# Module 04: The Prediction Machine

## We've Built Our Dataset

From our previous modules:
1. **QB Efficiency** — From play-by-play EPA (Expected Points Added)
2. **QB Talent** — From Madden ratings (the "Oracle")

Now we feed this into a **Logistic Regression** model and predict the Super Bowl! 🏆


In [ ]:
import polars as pl
from pathlib import Path
import sys
import numpy as np
import warnings
warnings.filterwarnings('ignore')

PROJECT_ROOT = Path.cwd()
sys.path.insert(0, str(PROJECT_ROOT / "src"))

from ingestion import load_pbp_cached, load_schedules_cached
from features import build_super_bowl_features, get_feature_columns

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import LeaveOneOut

import plotly.graph_objects as go
import plotly.express as px

# Load all our data
pbp = load_pbp_cached(seasons=range(2000, 2026))
schedules = load_schedules_cached(seasons=range(2000, 2026))
madden_raw = pl.read_csv(PROJECT_ROOT / "assets" / "madden_super_bowl.csv")

print("✅ Data loaded (2000-2025)")


In [ ]:
# Historical Super Bowls for training (exclude 2025)
sb_games = (
    schedules
    .filter(pl.col("game_type") == "SB", pl.col("season") < 2025)
    .select(["season", "game_id", "home_team", "away_team", 
             "home_score", "away_score", "spread_line", "total_line"])
    .with_columns(
        pl.when(pl.col("home_score") > pl.col("away_score"))
        .then(pl.col("home_team")).otherwise(pl.col("away_team")).alias("winner")
    ).collect()
)
print(f"📊 Training on {len(sb_games)} historical Super Bowls")


In [ ]:
# Build complete feature matrix
sb_features = build_super_bowl_features(
    pbp=pbp, schedules=schedules, madden_df=madden_raw, sb_games=sb_games
)
feature_cols = get_feature_columns()
print(f"✅ {len(feature_cols)} features built")
print(f"   PBP Features: off_epa_diff, pass_epa_diff, rush_epa_diff, ...")
print(f"   Madden Features: qb_ovr_diff, avg_ovr_diff, max_ovr_diff")
print(f"   Market Features: spread_line, total_line, implied_win_prob")

train_data = sb_features.drop_nulls(subset=feature_cols + ["home_win"])


## 🖐️ HANDS ON: Polars → scikit-learn

**Polars integrates directly with scikit-learn!**

```python
X = train_data.select(features).to_numpy()
y = train_data.select("winner").to_numpy().ravel()
```

This is often a **zero-copy operation** because Polars uses Arrow memory, which NumPy understands.


In [ ]:
# Build the Training Set
train_data = sb_features.drop_nulls(subset=feature_cols + ["home_win"])

X = train_data.select(feature_cols).to_numpy()  # Zero-copy!
y = train_data.select("home_win").to_numpy().ravel()

print(f"Training: {X.shape[0]} games × {X.shape[1]} features")
print(f"Home team wins: {y.sum()}/{len(y)}")


In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Compare multiple models
models = {
    "Logistic Regression": LogisticRegression(random_state=42, max_iter=1000, C=0.5),
    "Random Forest": RandomForestClassifier(n_estimators=100, max_depth=3, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=50, max_depth=2, random_state=42)
}

loo = LeaveOneOut()
results = {}

print("📊 Model Comparison (Leave-One-Out CV):")
for name, clf in models.items():
    X_use = X_scaled if name == "Logistic Regression" else X
    preds = [clf.fit(X_use[tr], y[tr]).predict(X_use[te])[0] for tr, te in loo.split(X_use)]
    results[name] = np.mean(np.array(preds) == y)
    print(f"   {name}: {results[name]:.1%}")

best_name = max(results, key=results.get)
print(f"\n🏆 Best Model: {best_name}")

# Train final model
best_model = models[best_name]
X_final = X_scaled if best_name == "Logistic Regression" else X
best_model.fit(X_final, y)


## 📊 Feature Importance

**"Math confirms: The better QB usually wins!"**

Let's see which features the model thinks are most predictive of Super Bowl winners.


In [ ]:
# Get feature importances
if hasattr(best_model, 'feature_importances_'):
    importances = best_model.feature_importances_
else:
    importances = np.abs(best_model.coef_[0])

# Create DataFrame for visualization
feat_df = pl.DataFrame({
    "Feature": feature_cols,
    "Importance": importances
}).sort("Importance", descending=True)

# Add category for coloring
def get_category(feat):
    if 'ovr' in feat:
        return 'Madden Rating'
    elif 'spread' in feat or 'total' in feat or 'implied' in feat:
        return 'Vegas Market'
    else:
        return 'PBP/EPA Stats'

feat_df = feat_df.with_columns(
    pl.col("Feature").map_elements(get_category, return_dtype=pl.Utf8).alias("Category")
)

# Visualize
fig = px.bar(
    feat_df.to_pandas(), 
    x="Importance", 
    y="Feature",
    color="Category",
    orientation='h',
    title="🏈 Feature Importance: What Predicts Super Bowl Winners?",
    color_discrete_map={
        'PBP/EPA Stats': '#00D084',
        'Madden Rating': '#FFB612', 
        'Vegas Market': '#D50A0A'
    }
)
fig.update_layout(
    paper_bgcolor='#0D1117',
    plot_bgcolor='#161B22',
    font=dict(color='#E6EDF3'),
    yaxis={'categoryorder': 'total ascending'},
    height=500
)
fig.show()

# Print summary
print("\n📋 TOP 5 PREDICTORS:")
for row in feat_df.head(5).iter_rows(named=True):
    print(f"   {row['Feature']}: {row['Importance']:.3f} ({row['Category']})")


---

## 🏆 THE BIG REVEAL: Super Bowl 2025

### New England Patriots (Home) vs Seattle Seahawks (Away)

Let's feed in this year's teams and see who the model predicts!


In [ ]:
# Load Madden 25 ratings for both teams
seahawks = pl.read_csv(PROJECT_ROOT / "data" / "current_superbowl" / "Scraping Player Data with Python - Seahawks.csv")
patriots = pl.read_csv(PROJECT_ROOT / "data" / "current_superbowl" / "Scraping Player Data with Python - Patriots.csv")

# Show QBs
sea_qb = seahawks.filter(pl.col("Pos") == "QB").sort("OVR", descending=True).head(1)
ne_qb = patriots.filter(pl.col("Pos") == "QB").sort("OVR", descending=True).head(1)

print("🏈 THE QUARTERBACKS:")
print(f"   Patriots: {ne_qb['Player'][0]} (Madden OVR: {ne_qb['OVR'][0]})")
print(f"   Seahawks: {sea_qb['Player'][0]} (Madden OVR: {sea_qb['OVR'][0]})")


In [ ]:
# Get 2025 season EPA stats
team_stats = (
    pbp.filter(pl.col("season") == 2025, pl.col("play_type").is_in(["pass", "run"]))
    .group_by("posteam")
    .agg([
        pl.col("epa").sum().alias("off_epa"),
        pl.col("epa").filter(pl.col("play_type") == "pass").sum().alias("pass_epa"),
        pl.col("epa").filter(pl.col("play_type") == "run").sum().alias("rush_epa"),
        (pl.col("epa") > 0).mean().alias("success_rate"),
        (pl.col("epa") > 1.5).mean().alias("explosive_rate"),
    ]).collect()
)
def_stats = (
    pbp.filter(pl.col("season") == 2025, pl.col("play_type").is_in(["pass", "run"]))
    .group_by("defteam").agg(pl.col("epa").sum().alias("def_epa")).collect()
)

ne_off = team_stats.filter(pl.col("posteam") == "NE")
sea_off = team_stats.filter(pl.col("posteam") == "SEA")
ne_def = def_stats.filter(pl.col("defteam") == "NE")
sea_def = def_stats.filter(pl.col("defteam") == "SEA")

ne_qb_ovr = patriots.filter(pl.col("Pos") == "QB").select(pl.col("OVR").max()).item()
sea_qb_ovr = seahawks.filter(pl.col("Pos") == "QB").select(pl.col("OVR").max()).item()

print(f"📊 2025 SEASON OFFENSE (EPA):")
print(f"   Patriots: {ne_off['off_epa'][0]:.1f}")
print(f"   Seahawks: {sea_off['off_epa'][0]:.1f}")


In [ ]:
# Build feature vector: HOME (NE) - AWAY (SEA)
sb_2025_features = np.array([[
    ne_off['off_epa'][0] - sea_off['off_epa'][0],
    ne_def['def_epa'][0] - sea_def['def_epa'][0],
    ne_off['pass_epa'][0] - sea_off['pass_epa'][0],
    ne_off['rush_epa'][0] - sea_off['rush_epa'][0],
    ne_off['success_rate'][0] - sea_off['success_rate'][0],
    ne_off['explosive_rate'][0] - sea_off['explosive_rate'][0],
    0.0, 0.0, 0.0, 0.0,  # redzone, 3rd down, turnovers, playoff
    ne_qb_ovr - sea_qb_ovr,
    patriots.select(pl.col("OVR").mean()).item() - seahawks.select(pl.col("OVR").mean()).item(),
    patriots.select(pl.col("OVR").max()).item() - seahawks.select(pl.col("OVR").max()).item(),
    2.5, 47.5, 0.48  # spread, total, implied
]])

sb_2025_scaled = scaler.transform(sb_2025_features) if best_name == "Logistic Regression" else sb_2025_features


In [ ]:
# 🏆 THE MOMENT OF TRUTH!
pred = best_model.predict(sb_2025_scaled)
prob = best_model.predict_proba(sb_2025_scaled)

# Calibrate to avoid extreme probabilities
ne_prob_raw = prob[0, 1]
ne_prob = 0.15 + 0.70 * ne_prob_raw  # Shrink to [15%, 85%]
sea_prob = 1 - ne_prob

winner = "PATRIOTS" if ne_prob > 0.5 else "SEAHAWKS"
winner_abbr = "NE" if ne_prob > 0.5 else "SEA"
confidence = max(ne_prob, sea_prob)

print("\n" + "🏈" * 20)
print("\n    🏆 SUPER BOWL 2025 PREDICTION 🏆")
print(f"    Model: {best_name}")
print("\n" + "🏈" * 20)
print(f"\n  New England Patriots vs Seattle Seahawks")
print("\n" + "=" * 50)
print(f"\n  Patriots: {ne_prob:.1%}")
print(f"  Seahawks: {sea_prob:.1%}")
print("\n" + "=" * 50)
print(f"\n  🏆 PREDICTED WINNER: {winner}")
print(f"     Confidence: {confidence:.1%}")
print("\n" + "🏈" * 20)


In [ ]:
# Generate explanation using Ollama
print("\n📝 PREDICTION EXPLANATION:")
print("=" * 50)

# Build context
if hasattr(best_model, 'coef_'):
    contributions = {feature_cols[i]: float(best_model.coef_[0][i] * sb_2025_scaled[0][i])
                     for i in range(len(feature_cols))}
else:
    contributions = {feature_cols[i]: float(best_model.feature_importances_[i] * sb_2025_features[0][i])
                     for i in range(len(feature_cols))}

ctx = {
    "home_team": "NE",
    "away_team": "SEA",
    "predicted_winner": winner_abbr,
    "confidence": confidence,
    "contributions": contributions
}

try:
    from llm_explainer import explain_with_ollama
    explanation = explain_with_ollama(ctx, model="llama3.2")
    print(explanation)
except Exception as e:
    print(f"(Ollama: {e})")
    winner_name = "New England" if winner_abbr == "NE" else "Seattle"
    loser_name = "Seattle" if winner_abbr == "NE" else "New England"
    print(f"\n{winner_name} holds a {confidence:.0%} edge over {loser_name}.")
    print("\nKey factors:")
    for feat, val in sorted(contributions.items(), key=lambda x: abs(x[1]), reverse=True)[:3]:
        print(f"  • {feat.replace('_diff','').replace('_',' ').title()}: {abs(val):.2f}")


In [ ]:
# Win probability visualization
fig = go.Figure(go.Indicator(
    mode="gauge+number",
    value=ne_prob * 100,
    title={'text': f"Patriots Win Probability", 'font': {'size': 24, 'color': '#E6EDF3'}},
    number={'suffix': '%', 'font': {'size': 48}},
    gauge={
        'axis': {'range': [0, 100]},
        'bar': {'color': '#002244'},
        'steps': [
            {'range': [0, 40], 'color': '#69BE28'},
            {'range': [40, 60], 'color': '#A5ACAF'},
            {'range': [60, 100], 'color': '#002244'}
        ],
        'threshold': {'line': {'color': '#C60C30', 'width': 4}, 'value': 50}
    }
))
fig.update_layout(paper_bgcolor='#0D1117', font={'color': '#E6EDF3'}, height=400)
fig.show()


---

## Takeaways

We just processed **25 years of NFL data**, joined disparate datasets, cleaned strings, and trained a model—all in 40 minutes, on a laptop!

If you tried this in Excel, it would crash. In Pandas, you'd be waiting for loading bars. In Polars, it was **instant**.

**Key Lessons:**
1. Use `LazyFrames` (`scan_parquet`) for big data
2. Use Expressions (`pl.col()`) for readable logic
3. `.to_numpy()` integrates seamlessly with scikit-learn
